Homotopy Continuation/Certfication file. This file contains all different versions of homotopy continuation as described in the thesis, defined as functions. It also contains functions for certification and coefficient-parameter homotopies.

First, we import the necessary packages including the file containing the data for our examples.

In [1]:
import random
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from mpmath import iv
import Data_for_Examples as E

We define all functions that are used repeatedly. This does not include functions such as Euler's and Newton's method, since we adjust these along the way.

In [2]:
#Find variables of function:
def Var(F):
    # If F only conists of one function, we put it in list, so we can iterate over it:
    if not isinstance(F, tuple):
        F = [F]
    # Fing all unique sumpy symbols from each function:
    variables = set()
    for f in F:
        variables.update(f.free_symbols)
    # Sort the variables to ensure consistent ordering i.e. x1=x1,x2=x2...
    variables = sorted(variables, key=lambda x: x.name)
    # Add the additional variable 't'
    variables.append(sp.symbols('t'))
    # Convert to tuple
    variables = tuple(variables) 
    return variables #Returns a tuple of variables including t

#Total degree start system of polynomial system F:
def Total_degree_start_system(variables,F):
    # Calculate total degree for each polynomial in the system
    total_degrees = []
    if len(variables[:-1])==1: #Special case if there is only one polynomial
        total_degree=max(F.as_poly(variables).monoms()[0])
        total_degrees.append(total_degree)
        G=np.array(variables[0]**total_degree-1)
    else: #Generel case:
        for poly in F:
            total_degree = max(sum(mon) for mon in poly.as_poly(variables).monoms()) #Sum over the degrees with respect to each variable in each term, and find the maximum.
            total_degrees.append(total_degree)
        #Find starting polynomial system G of total degree:
        G=np.array([])
        for i in range(len(variables[:-1])): #Iterate over variables (except t)
            G=np.append(G,variables[i]**total_degrees[i]-1) #Set each variable to the total degree of the corresponding polynomial and subtract one.
    return G,total_degrees

def Homogenize_function(variables,F):    
    x0=sp.symbols('x0')
    F_homogeneous = np.array([])
    #If we have one polynomial of one variable:
    if len(variables[:-1])==1:
        hom_poly=0
        total_degree=max(F.as_poly(variables).monoms()[0]) #Sum over the degrees with respect to each variable in each term, and find the maximum.
        for term in F.as_terms()[0]:#Iterate over the terms in the polynomial F
            term_degree=sum(term[0].as_poly(variables).monoms()[0]) #Find the degree of the terms
            hom_term=term[0]*x0**(total_degree-term_degree) #Multiply term by x0 raised to a power corresponding to the difference of the term and the total degree
            hom_poly+=hom_term #Construct a homogeneous polynomial consisting of each of the homogenized terms.
        F_homogeneous=np.append(F_homogeneous,hom_poly) 
    else:  #If we have multiple polynomials and variables:
        for poly in F: #Iterate over the polynomials in F
            hom_poly=0
            total_degree = max(sum(mon) for mon in poly.as_poly(variables).monoms()) #Sum over the degrees with respect to each variable in each term, and find the maximum.
            for term in poly.as_terms()[0]: #Iterate over the terms in the polynomial
                term_degree=sum(term[0].as_poly(variables).monoms()[0]) #Find the degree of the terms
                hom_term=term[0]*x0**(total_degree-term_degree) #Multiply term by x0 raised to a power corresponding to the difference of the term and the total degree
                hom_poly+=hom_term #Construct a homogeneous polynomial consisting of each of the homogenized terms.
            F_homogeneous=np.append(F_homogeneous,hom_poly) #Collect all homogenized polynomials
    return F_homogeneous

#Function that homogenizes a polynomial to a degree corresponding to the specififed start system, which may not be a total degree start system.
def Homogenize_with_respect_to_total_degree_of_G(variables, F, G):
    x0=sp.symbols('x0')
    F_homogeneous = np.array([])
    #If we have one polynomial of one variable:
    if len(variables[:-1])==1:
        hom_poly=0
        total_degree=max(G.as_poly(variables).monoms()[0]) #Sum over the degrees with respect to each variable in each term, and find the maximum.
        for term in F.as_terms()[0]:#Iterate over the terms in the polynomial F
            term_degree=sum(term[0].as_poly(variables).monoms()[0]) #Find the degree of the terms
            hom_term=term[0]*x0**(total_degree-term_degree) #Multiply term by x0 raised to a power corresponding to the difference of the term and the total degree
            hom_poly+=hom_term #Construct a homogeneous polynomial consisting of each of the homogenized terms.
        F_homogeneous=np.append(F_homogeneous,hom_poly) 
    else:  #If we have multiple polynomials and variables:
        for i in range(len(F)): #Iterate over the polynomials in F
            poly=F[i]
            hom_poly=0
            total_degree = max(sum(mon) for mon in G[i].as_poly(variables).monoms()) #Sum over the degrees with respect to each variable in each term, and find the maximum.
            for term in poly.as_terms()[0]: #Iterate over the terms in the polynomial
                term_degree=sum(term[0].as_poly(variables).monoms()[0]) #Find the degree of the terms
                hom_term=term[0]*x0**(total_degree-term_degree) #Multiply term by x0 raised to a power corresponding to the difference of the term and the total degree
                hom_poly+=hom_term #Construct a homogeneous polynomial consisting of each of the homogenized terms.
            F_homogeneous=np.append(F_homogeneous,hom_poly) #Collect all homogenized polynomials
    return F_homogeneous

def Homogenized_total_degree_start_system(variables,F):
    #Follows same strategy as the function Total_degree_start_system.
    # Calculate total degree for each polynomial in the system
    x0=sp.symbols('x0')
    total_degrees = []
    if len(variables[:-1])==1: #Special case if there is only one polynomial
        total_degree=max(F.as_poly(variables).monoms()[0])
        total_degrees.append(total_degree)
        G=np.array(variables[0]**total_degree-x0**total_degree)
    else: #Generel case:
        for poly in F:
            total_degree = max(sum(mon) for mon in poly.as_poly(variables).monoms()) #Sum over the degrees with respect to each variable in each term, and find the maximum.
            total_degrees.append(total_degree)
        #Find starting polynomial system G of total degree:
        G=np.array([])
        for i in range(len(variables[:-1])):
            G=np.append(G,variables[i]**total_degrees[i]-x0**total_degrees[i]) #Set each variable to the total degree of the corresponding polynomial and subtract x0 raised to the toal degree.
    return G,total_degrees

#def Function_from_parameters(q,variables,F):
#    q_F=np.array([])
#    for i in range(len(F)): #Iterate over the polynomials in F
#        poly=F[i]
#        q_poly=0
#        for j in range(len(poly.as_terms()[0][0])): #Iterate over the terms in the polynomial
#            print(poly.as_terms()[0][0])
#            term=(poly.as_terms()[0][0])[j]
#            print(term)
#            q_term=term*q[i][j]
#            print(q_term)
#            q_poly+=q_term #Construct a homogeneous polynomial consisting of each of the homogenized terms.
#        q_F.append(q_F_,q_poly) #Collect all homogenized polynomials
#    return q_F

First version of Homotopy continuation, corresponding to Algorithm 3.3.1:

In [3]:
#First version of Homotopy Continuation:
def Homotopy_Continuation_1(F,M=100,specified_gamma=None):
    #Define variables, and define x as variables that does not include t:
    variables=Var(F)
    t=variables[-1]
    x=variables[:-1]

    #Total degree start system G along with list of total degrees: 
    G,degrees=Total_degree_start_system(variables,F)
    
    #Set function in array for syntax:
    F=np.array(F)

    #Set stepsize h:
    h=1.0/M

    #Generate a random complex number gamma of modulus 1:
    if specified_gamma is None:
        c_number=complex(random.uniform(-1,1),random.uniform(-1,1))
        gamma=c_number/np.linalg.norm(c_number)
    else:
        gamma=specified_gamma

    #This defines our homotopy and the functions generated by the Davidenko equations:
    H=t*F+gamma*(1-t)*G #Homotopy H
    dtH = sp.lambdify(variables,sp.Matrix([H]).diff(t),'numpy') #Derivative of H with respect to t, which can be evaluated at a point
    dxH = sp.lambdify(variables,sp.Matrix([H]).jacobian([x]),'numpy') #Derivatibe of H with respect to x, which can be evaluated at a point
    H_eval=sp.lambdify(variables,H,'numpy') #H, which can be evaluated at a point

    #Define Euler step:
    def euler_method(x_euler,t_euler):
        return x_euler-h * (np.linalg.inv(dxH(*(x_euler[:,0]),t_euler)) @ np.transpose(dtH(*(x_euler[:,0]),t_euler))) #Definition from the Davidenko equations.
    
    #Define Newton step:
    def newton_method(x_newton, t_newton, tol=1e-6,max_iterations=3):
        k=0
        H_newton=np.array(H_eval(*np.append((x_newton[:,0]),[t_newton]))).reshape(-1,1) #Initial value of H
        while any(abs(a) > tol for a in H_newton) and k<max_iterations: #While not convergent, meaning modulus of each entry is large and we have used fewer than "max_iterations" iterations.
            x_newton-= (np.linalg.inv(dxH(*x_newton[:,0], t_newton))@H_newton) #Update solutions
            H_newton=np.array(H_eval(*np.append((x_newton[:,0]),[t_newton]))).reshape(-1,1) #Update H
            k+=1
        return x_newton        
    
    #Define function that constructs a path for each solution between F and G:
    def path_solution(x0_single, max_change=100):
        plot_points=[x0_single] #Keep track of points we want to plot
        j=0
        while j <M:  # Iterate through each step
            x0_prev=np.copy(x0_single) #Save previous point.
            t_j = j*h
            # Predictor step (Euler's method)        
            x_pc = euler_method(x0_single, t_j)
            # Corrector step (Newton's method)
            x0_single = newton_method(x_pc, t_j+h)
            plot_points.append(x0_single) #Keep track of points we want to plot
            
            #If the difference between the current and previous point is too large, path goes to infinity.
            if np.linalg.norm(x0_single-x0_prev) > max_change*1/M:
                return "Path is going to infinity", plot_points    
            j+=1
        return x0_single, plot_points

    #Finding roots of G:
    x0=[]    
    roots_initial=[np.exp(2j * np.pi * np.arange(d) / d) for d in degrees] #Find roots of unity with respect to each of the degrees in the startsystem.
    #We find all combinations of solutions, which is our solutions to the total degree start system.
    for root0 in roots_initial[0]:
        x0.append([root0])
    for roots in roots_initial[1:]:
        np.ndarray.tolist(roots)
        x0=[a+[b] for a in x0 for b in roots] #We construct a list of lists consisting of the solutions of G.

    #Construct path for each of the initial solutions G to target solutions F:
    x_new=[]
    plot_x=[] 
    for x0_sol in x0:
        x0_sol,plot_sol=path_solution(np.array(x0_sol).reshape(-1,1))
        x_new.append(x0_sol)
        plot_x.append(plot_sol)
    
    #Plot solution paths:
    fig, ax = plt.subplots()
    for xi_plotlist in plot_x:
        # Extract real and imaginary parts of the solutions
        real_parts = [x.real for x in np.squeeze(xi_plotlist)]
        imag_parts = [x.imag for x in np.squeeze(xi_plotlist)]
        # Plot the path for the current root
        color = next(ax._get_lines.prop_cycler)['color'] #Ensures end points has same color as path
        ax.plot(real_parts, imag_parts, marker=',', linestyle='-',color=color)
        ax.plot(real_parts[0], imag_parts[0], marker='s', linestyle='-',color=color) #Path starts with a square.
        ax.plot(real_parts[-1], imag_parts[-1], marker='x', linestyle='-',color=color) #Path ends in an asterix.
    ax.set_xlabel('Real Part')
    ax.set_ylabel('Imaginary Part')
    ax.set_title(f'Solution Paths for the Homotopy h: {H}')
    plt.grid(True)
    plt.show()    

    #Return solutions
    Solution_list=[]
    for i in range(len(x_new)):
        Solution_list.append(f"Solution {i+1}: {x_new[i]}")
    return Solution_list

Second version of homotopy continuation, computing list of solutions in each iteration of the path:

In [4]:
#Second version of Homotopy Continuation:
def Homotopy_Continuation_2_table(F, tol, max_iterations, M=10,specified_gamma=None):
    #Define variables, and define x as variables that does not include t:
    variables=Var(F)
    t=variables[-1]
    x=variables[:-1]
    
    #Total degree start system G along with list of total degrees: 
    G,degrees=Total_degree_start_system(variables,F)
    
    #Set function in array for syntax:
    F=np.array(F)

    #Set stepsize h:
    h=1.0/M

    #Generate a random complex number gamma of modulus 1:
    if specified_gamma is None:
        c_number=complex(random.uniform(-1,1),random.uniform(-1,1))
        gamma=c_number/np.linalg.norm(c_number)
    else:
        gamma=specified_gamma

    #This defines our homotopy and the functions generated by the Davidenko equations:
    H=t*F+gamma*(1-t)*G #Homotopy H
    dtH = sp.lambdify(variables,sp.Matrix([H]).diff(t),'numpy') #Derivative of H with respect to t, which can be evaluated at a point
    dxH = sp.lambdify(variables,sp.Matrix([H]).jacobian([x]),'numpy') #Derivatibe of H with respect to x, which can be evaluated at a point
    H_eval=sp.lambdify(variables,H,'numpy') #H, which can be evaluated at a point

    #Define Euler step:
    def euler_method(x_euler,t_euler):
        return x_euler-h * (np.linalg.inv(dxH(*(x_euler[:,0]),t_euler)) @ np.transpose(dtH(*(x_euler[:,0]),t_euler))) #Definition from the Davidenko equations.
    
    #Define Newton step:
    def newton_method(x_newton, t_newton, tol, max_iterations):
        k=0
        H_newton=np.array(H_eval(*np.append((x_newton[:,0]),[t_newton]))).reshape(-1,1) #Initial value of H
        while any(abs(a) > tol for a in H_newton) and k<max_iterations: #While not convergent, meaning modulus of each entry is large and we have used fewer than "max_iterations" iterations.
            x_newton-= (np.linalg.inv(dxH(*x_newton[:,0], t_newton))@H_newton) #Update solutions
            H_newton=np.array(H_eval(*np.append((x_newton[:,0]),[t_newton]))).reshape(-1,1) #Update H
            k+=1
        return x_newton
    
    #Define function that constructs a path for each solution between F and G:
    def path_solution(x0_single, max_change=100):
        plot_points=[x0_single] #Keep track of points we want to plot
        j=0
        while j <M:  # Iterate through each step
            x0_prev=np.copy(x0_single) #Save previous point.
            t_j = j*h
            # Predictor step (Euler's method)        
            x_pc = euler_method(x0_single, t_j)
            # Corrector step (Newton's method)
            x0_single= newton_method(x_pc, t_j+h,tol,max_iterations)
            plot_points.append(x0_single) #Keep track of points we want to plot
            
            #If the difference between the current and previous point is too large, path goes to infinity.
            if np.linalg.norm(x0_single-x0_prev) > max_change*1/M:
                return "Path is going to infinity", plot_points    
            j+=1
        return x0_single, plot_points

    #Finding roots of G:
    x0=[]    
    roots_initial=[np.exp(2j * np.pi * np.arange(d) / d) for d in degrees] #Find roots of unity with respect to each of the degrees in the startsystem.
    #We find all combinations of solutions, which is our solutions to the total degree start system.
    for root0 in roots_initial[0]:
        x0.append([root0])
    for roots in roots_initial[1:]:
        np.ndarray.tolist(roots)
        x0=[a+[b] for a in x0 for b in roots] #We construct a list of lists consisting of the solutions of G.

    #Construct path for each of the initial solutions G to target solutions F:
    x_new=[]
    plot_x=[]
    for x0_sol in x0:
        x0_sol,plot_sol=path_solution(np.array(x0_sol).reshape(-1,1))
        x_new.append(x0_sol)
        plot_x.append(plot_sol)

    #Return solutions and intermetdiate solutions at each step:
    Solution_list=[]
    for i in range(len(x_new)):
        Solution_list.append(f"Solution {i+1}: {x_new[i]}, Intermediate solutions at each iteration: {plot_x[i]}")
#        Intermediate_sol_list.append(f"Intermediate solutions at each iteration: {plot_x[i]}")
    return Solution_list

Third version of homotopy continuation, where we use one newton iteration per step for a strict tolerance of $10^{-12}$:

In [5]:
#Third version of Homotopy Continuation:
def Homotopy_Continuation_3(F,M=100,specified_gamma=None):
    #Define variables, and define x as variables that does not include t:
    variables=Var(F)
    t=variables[-1]
    x=variables[:-1]

    #Total degree start system G along with list of total degrees: 
    G,degrees=Total_degree_start_system(variables,F)
    
    #Set function in array for syntax:
    F=np.array(F)

    #Set stepsize h:
    h=1.0/M

    #Generate a random complex number gamma of modulus 1:
    if specified_gamma is None:
        c_number=complex(random.uniform(-1,1),random.uniform(-1,1))
        gamma=c_number/np.linalg.norm(c_number)
    else:
        gamma=specified_gamma

    #This defines our functions generated by the Davidenko equations
    H=t*F+gamma*(1-t)*G
    dtH = sp.lambdify(variables,sp.Matrix([H]).diff(t),'numpy')
    dxH = sp.lambdify(variables,sp.Matrix([H]).jacobian([x]),'numpy')
    H_eval=sp.lambdify(variables,H,'numpy')
    def euler_method(x_euler,t_euler,h):
        return x_euler-h * (np.linalg.inv(dxH(*(x_euler[:,0]),t_euler)) @ np.transpose(dtH(*(x_euler[:,0]),t_euler))) 
    def newton_method(x_newton, t_newton, tol,max_iterations):
        k=0
        H_newton=np.array(H_eval(*np.append((x_newton[:,0]),[t_newton]))).reshape(-1,1) #Initial value of H
        while any(np.linalg.norm(a) > tol for a in H_newton) and k<max_iterations: #While not convergent
            x_newton-= (np.linalg.inv(dxH(*x_newton[:,0], t_newton))@H_newton) #Update solutions
            H_newton=np.array(H_eval(*np.append((x_newton[:,0]),[t_newton]))).reshape(-1,1) #Update H
            k+=1
        if k == max_iterations: #Newton step converges if the step is within theshold of true solution within the maximum number of iterations.
            convergence=False
        else:
            convergence=True
        return x_newton, convergence,k
    def path_solution(x0_single, stepsize=1.0/M, max_change=1000):
        plot_points=[x0_single]
        h=stepsize
        stepsize_end=stepsize/2
        t_j=0
        steps=0
        newton_iterations=0
        consecutive_convergences = 0    
        while t_j+h<1:  # Iterate through each step
            x0_prev=np.copy(x0_single)        
            # Predictor step (Euler's method)        
            x_pc = euler_method(x0_single, t_j,h)
            t_j+=h
            # Corrector step (Newton's method)
            if t_j>0.9: #Near the end we have a stronger treshold for convergence and more possible iterations.
                x0_single, convergence,k = newton_method(x_pc, t_j,tol=1e-2,max_iterations=3)
                stepsize=stepsize_end
                newton_iterations+=k            
            else: #In most of the path we have a weak treshold and few iterations.
                x0_single, convergence,k = newton_method(x_pc, t_j,tol=1e-1,max_iterations=1)
                newton_iterations+=k
            plot_points.append(x0_single)          
            if np.linalg.norm(x0_single-x0_prev) > max_change*h: #How do we deal with paths going to infinity?
                return "Path is going to infinity", plot_points, steps, newton_iterations    
            if convergence is True: #Checks if newton step converges or not.
                consecutive_convergences += 1
                if consecutive_convergences >= 2:
                    h=2*stepsize  # Double the step size after 2 consecutive convergences
                else:
                    h=stepsize   
            else:
                h=stepsize/2  # Halve the step size if not converged
                consecutive_convergences = 0  # Reset counter
            steps+=1        
        if t_j+h>=1: #Final step.
            h=1-t_j #Set h such that we finish at t=1.
            x0_prev=np.copy(x0_single)
            x_pc = euler_method(x0_single, t_j,h)
            x0_single = newton_method(x_pc, 1,tol=1e-12,max_iterations=10)[0]
            plot_points.append(x0_single)
            steps+=1
            newton_iterations+=k
            if np.linalg.norm(x0_single-x0_prev) > max_change*h: #How do we deal with paths going to infinity
                return "Path is going to infinity", plot_points, steps, newton_iterations
        return x0_single, plot_points, steps, newton_iterations

    #Finding roots of G:
    roots_initial=[np.exp(2j * np.pi * np.arange(d) / d) for d in degrees]
    x0=[]
    #This gives us all combinations of solutions to our initial problem:
    for root0 in roots_initial[0]:
        x0.append([root0])
    for roots in roots_initial[1:]:
        np.ndarray.tolist(roots)
        x0=[a+[b] for a in x0 for b in roots]
    x_new=[]
    plot_x=[]
    steps_list=[]
    NI_list=[]

    #Find solution paths for all initial solutions:
    for x0_sol in x0:
        x0_sol,plot_sol,steps, newton_iterations=path_solution(np.array(x0_sol).reshape(-1,1))
        x_new.append(x0_sol)
        plot_x.append(plot_sol)
        steps_list.append(steps)
        NI_list.append(newton_iterations)

    #Return solutions, steps and number of newton iterations:
    Solution_list=[]
    for i in range(len(x_new)):
        Solution_list.append(f"Solution {i+1}: {x_new[i]}, Number of steps: {steps_list[i]}, Number of newton iterations: {NI_list[i]}")               
    return Solution_list

Fouth version of homotopy continuation, using Runge-Kutta instead og Euler's method in Algorithm 4.1.1.

In [6]:
#Fourth version of Homotopy Continuation:
def Homotopy_Continuation_4(F,M=100,specified_gamma=None):
    #Define variables, and define x as variables that does not include t:
    variables=Var(F)
    t=variables[-1]
    x=variables[:-1]
    
    #Total degree start system G along with list of total degrees: 
    G,degrees=Total_degree_start_system(variables,F)
    
    #Set function in array for syntax:
    F=np.array(F)

    #Set stepsize h:
    h=1.0/M

    #Set x for variables that does not include t:
    t=variables[-1]
    x=variables[:-1]

    #Generate a random complex number gamma of modulus 1:
    if specified_gamma is None:
        c_number=complex(random.uniform(-1,1),random.uniform(-1,1))
        gamma=c_number/np.linalg.norm(c_number)
    else:
        gamma=specified_gamma

    #This defines our functions generated by the Davidenko equations
    H=t*F+gamma*(1-t)*G
    dtH = sp.lambdify(variables,sp.Matrix([H]).diff(t),'numpy')
    dxH = sp.lambdify(variables,sp.Matrix([H]).jacobian([x]),'numpy')
    H_eval=sp.lambdify(variables,H,'numpy')
    
    #Funnction that defines Delta x:
    def scipy_delta_x(x,t):
        return -(np.linalg.inv(dxH(*(x[:,0]),t)) @ np.transpose(dtH(*(x[:,0]),t))) 
    #Runge-Kutta step:
    def runge_kutta(x_rk,t_rk,h):
        k1=scipy_delta_x(x_rk,t_rk)
        k2=scipy_delta_x(x_rk+(h/2)*k1,t_rk+(h/2))
        k3=scipy_delta_x(x_rk+(h/2)*k2,t_rk+(h/2))
        k4=scipy_delta_x(x_rk+h*k3,t_rk+h)
        return x_rk+(h/6)*(np.array(k1).reshape(-1,1)+2*np.array(k2).reshape(-1,1)+2*np.array(k3).reshape(-1,1)+np.array(k4).reshape(-1,1))
    def newton_method(x_newton, t_newton, tol=1e-6,max_iterations=3):
        k=0
        H_newton=np.array(H_eval(*np.append((x_newton[:,0]),[t_newton]))).reshape(-1,1) #Initial value of H
        while any(np.linalg.norm(a) > tol for a in H_newton) and k<max_iterations: #While not convergent
            x_newton-= (np.linalg.inv(dxH(*x_newton[:,0], t_newton))@H_newton) #Update solutions
            H_newton=np.array(H_eval(*np.append((x_newton[:,0]),[t_newton]))).reshape(-1,1) #Update H
            k+=1
        if k == max_iterations:
            convergence=False
        else:
            convergence=True
        return x_newton, convergence,k
    def path_solution(x0_single, stepsize=1.0/M, max_change=1000):
        plot_points=[x0_single]
        h=stepsize
        stepsize_end=stepsize/2
        t_j=0
        steps=0
        newton_iterations=0
        consecutive_convergences = 0    
        while t_j+h<1:  # Iterate through each step
            x0_prev=np.copy(x0_single)        
            # Predictor step (Euler's method)        
            x_pc = runge_kutta(x0_single, t_j,h)
            t_j+=h
            steps+=1 
            # Corrector step (Newton's method)
            if t_j>0.9:
                x0_single, convergence,k = newton_method(x_pc, t_j,tol=1e-9,max_iterations=3)
                stepsize=stepsize_end
                newton_iterations+=k            
            else:
                x0_single, convergence,k = newton_method(x_pc, t_j,tol=1e-6,max_iterations=1)
                newton_iterations+=k
            plot_points.append(x0_single)
            if np.linalg.norm(x0_single-x0_prev) > max_change*h: #How do we deal with paths going to infinity?
                return "Path is going to infinity", plot_points,steps,newton_iterations    
            if convergence is True:
                consecutive_convergences += 1
                if consecutive_convergences >= 2:
                    h=2*stepsize  # Double the step size after 2 consecutive convergences
                else:
                    h=stepsize   
            else:
                h=stepsize/2  # Halve the step size if not converged
                consecutive_convergences = 0  # Reset counter        
        if t_j+h>=1:
            h=1-t_j
            x0_prev=np.copy(x0_single)
            x_pc = runge_kutta(x0_single, t_j,h)
            x0_single, convergence,k = newton_method(x_pc, 1,tol=1e-12,max_iterations=10)
            plot_points.append(x0_single)
            steps+=1
            newton_iterations+=k
            if np.linalg.norm(x0_single-x0_prev) > max_change*h: #How do we deal with paths going to infinity
                return "Path is going to infinity", plot_points,steps,newton_iterations
        return x0_single, plot_points, steps, newton_iterations

    #Finding roots of G:
    roots_initial=[np.exp(2j * np.pi * np.arange(d) / d) for d in degrees]
    x0=[]
    #This gives us all combinations of solutions to our initial problem:
    for root0 in roots_initial[0]:
        x0.append([root0])
    for roots in roots_initial[1:]:
        np.ndarray.tolist(roots)
        x0=[a+[b] for a in x0 for b in roots]
    x_new=[]
    plot_x=[]
    steps_list=[]
    NI_list=[]
    for x0_sol in x0:
        x0_sol,plot_sol,steps, newton_iterations=path_solution(np.array(x0_sol).reshape(-1,1))
        x_new.append(x0_sol)
        plot_x.append(plot_sol)
        steps_list.append(steps)
        NI_list.append(newton_iterations)

    #Return solutions, steps and number of newton iterations:
    Solution_list=[]
    Solution_list_data=[]
    for i in range(len(x_new)):
        Solution_list.append(f"Solution {i+1}: {x_new[i]}, Number of steps: {steps_list[i]}, Number of newton iterations: {NI_list[i]}")               
        if not isinstance(x_new[i],str):
            Solution_list_data.append(x_new[i])        
    return Solution_list, Solution_list_data

Fifth version of Homotopy Continuation using Algorithm 4.2.1.

In [7]:
#Fifth version of Homotopy Continuation:
def Homotopy_Continuation_5(F,M=100,specified_gamma=None):
    #Define variables, and define x as variables that does not include t:
    variables=Var(F)
    t=variables[-1]
    x=variables[:-1]

    #Total degree start system G along with list of total degrees: 
    G,degrees=Total_degree_start_system(variables,F)
    
    #Set function in array for syntax:
    F=np.array(F)

    #Set stepsize h:
    h=1.0/M

    #Generate a random complex number gamma of modulus 1:
    if specified_gamma is None:
        c_number=complex(random.uniform(-1,1),random.uniform(-1,1))
        gamma=c_number/np.linalg.norm(c_number)
    else:
        gamma=specified_gamma

    #This defines our functions generated by the Davidenko equations
    H=t*F+gamma*(1-t)*G
    dtH = sp.lambdify(variables,sp.Matrix([H]).diff(t),'numpy')
    dxH = sp.lambdify(variables,sp.Matrix([H]).jacobian([x]),'numpy')
    H_eval=sp.lambdify(variables,H,'numpy')

    def scipy_delta_x(x,t):
        return -(np.linalg.inv(dxH(*(x[:,0]),t)) @ np.transpose(dtH(*(x[:,0]),t)))
    def euler_method(x_euler,t_euler,h):
        return x_euler+h * scipy_delta_x(x_euler,t_euler)
    def runge_kutta(x_rk,t_rk,h):
        k1=scipy_delta_x(x_rk,t_rk)
        k2=scipy_delta_x(x_rk+(h/2)*k1,t_rk+(h/2))
        k3=scipy_delta_x(x_rk+(h/2)*k2,t_rk+(h/2))
        k4=scipy_delta_x(x_rk+h*k3,t_rk+h)
        return x_rk+(h/6)*(np.array(k1).reshape(-1,1)+2*np.array(k2).reshape(-1,1)+2*np.array(k3).reshape(-1,1)+np.array(k4).reshape(-1,1))
    #Explicit adams_bashfort method using information about previous steps.
    def adams_bashfort_method(x_ab0,x_ab1,x_ab2,x_ab3,t_ab0,t_ab1,t_ab2,t_ab3,h):
        x_ab0_diff=scipy_delta_x(x_ab0,t_ab0)
        x_ab1_diff=scipy_delta_x(x_ab1,t_ab1)
        x_ab2_diff=scipy_delta_x(x_ab2,t_ab2)
        x_ab3_diff=scipy_delta_x(x_ab3,t_ab3)
        return x_ab3+h/24*(55*x_ab3_diff-59*x_ab2_diff+37*x_ab1_diff-9*x_ab0_diff)
    #Implicit adams_moulton method using information about previous steps and derivative of predicted step.
    def adams_moulton_method(x_ab0,x_ab1,x_ab2,x_ab3,t_ab0,t_ab1,t_ab2,t_ab3,h):
        x_ab0_diff=scipy_delta_x(x_ab0,t_ab0)
        x_ab1_diff=scipy_delta_x(x_ab1,t_ab1)
        x_ab2_diff=scipy_delta_x(x_ab2,t_ab2)
        x_ab3_diff=scipy_delta_x(x_ab3,t_ab3)
        return x_ab2+(h/24)*(9*x_ab3_diff+19*x_ab2_diff-5*x_ab1_diff+x_ab0_diff)
    #Combined method:
    def abam_method(x_ab0,x_ab1,x_ab2,x_ab3,t_ab0,t_ab1,t_ab2,t_ab3,h):
        x_forward=adams_bashfort_method(x_ab0,x_ab1,x_ab2,x_ab3,t_ab0,t_ab1,t_ab2,t_ab3,h)
        x_backward=adams_moulton_method(x_ab1,x_ab2,x_ab3,x_forward,t_ab1,t_ab2,t_ab3,t_ab3+h,h)
        return x_backward
    def newton_method(x_newton, t_newton, tol=1e-6,max_iterations=3):
        k=0
        H_newton=np.array(H_eval(*np.append((x_newton[:,0]),[t_newton]))).reshape(-1,1) #Initial value of H
        while any(np.linalg.norm(a) > tol for a in H_newton) and k<max_iterations: #While not convergent
            x_newton-= (np.linalg.inv(dxH(*x_newton[:,0], t_newton))@H_newton) #Update solutions
            H_newton=np.array(H_eval(*np.append((x_newton[:,0]),[t_newton]))).reshape(-1,1) #Update H
            k+=1
        if k == max_iterations:
            convergence=False
        else:
            convergence=True
        return x_newton, convergence,k

    def path_solution(x0_single, stepsize=1.0/M, max_change=1000):    
        plot_points=[x0_single[1]]
        x_list=[x0_single]
        h=stepsize
        t_j=0
        j=0
        steps_euler=0
        steps_abam=0
        newton_iterations=0
        for j in range(0,M):  # Iterate through each step
            t_j = j * h
            max_eig,min_eig=max(abs(np.linalg.eigvals(dxH(*scipy_delta_x(x0_single,t_j)[:,0],t_j)))),min(abs(np.linalg.eigvals(dxH(*scipy_delta_x(x0_single,t_j)[:,0],t_j)))) #Largest and smallest eignevalues of Jacobian of change of x with respect to absolute value
            if j<3: #First three steps use Runge-Kutta.
                x_pc=runge_kutta(x0_single,t_j,h)
            else:
                if max_eig<10 and max_eig/min_eig<10: #Checks if the difference is relatively large
                    x_pc=euler_method(x0_single,t_j,h) #Use Euler as predictor when equation is non stiff
                    steps_euler+=1            
                else:
                    x_pc=abam_method(x_list[j-3],x_list[j-2],x_list[j-1],x_list[j],t_j-3*h,t_j-2*h,t_j-h,t_j,h) #Use abam predictor for stiff problems
                    steps_abam+=1
            #Newton method varying depending on value of t.
            if 1>t_j>0.9:
                x0_single,convergence,k=newton_method(x_pc, t_j+h,tol=1e-9,max_iterations=3)
                newton_iterations+=k
            elif j==(M-1):
                x0_single,convergence,k = newton_method(x_pc,1,tol=1e-12,max_iterations=10)
                newton_iterations+=k
            else:
                x0_single,convergence,k=newton_method(x_pc, t_j+h,tol=1e-6,max_iterations=1)                
                newton_iterations+=k
            x_list.append(x0_single)
            if np.linalg.norm(x0_single-x_list[j]) > max_change*h: #How do we deal with paths going to infinity
                return "Path is going to infinity", plot_points,steps_euler,steps_abam,newton_iterations
        return x0_single, plot_points,steps_euler, steps_abam,newton_iterations

    #Finding roots of G:
    roots_initial=[np.exp(2j * np.pi * np.arange(d) / d) for d in degrees]
    x0=[]
    #This gives us all combinations of solutions to our initial problem:
    for root0 in roots_initial[0]:
        x0.append([root0])
    for roots in roots_initial[1:]:
        np.ndarray.tolist(roots)
        x0=[a+[b] for a in x0 for b in roots]
    x_new=[]
    plot_x=[]
    steps_list_euler=[]
    steps_list_abam=[]
    NI_list=[]
    for x0_sol in x0:
        x0_sol,plot_sol,steps_euler,steps_abam, newton_iterations=path_solution(np.array(x0_sol).reshape(-1,1))
        x_new.append(x0_sol)
        plot_x.append(plot_sol)
        steps_list_euler.append(steps_euler)
        steps_list_abam.append(steps_abam)
        NI_list.append(newton_iterations)

    #Return solutions, steps and number of newton iterations:
    Solution_list=[]
    Solution_list_data=[]
    for i in range(len(x_new)):
        Solution_list.append(f"Solution {i+1}: {x_new[i]}, Number of Euler steps: {steps_list_euler[i]}, Number of ABAM steps: {steps_list_abam[i]}, Number of newton iterations: {NI_list[i]}")               
        if not isinstance(x_new[i],str):
            Solution_list_data.append(x_new[i])
    return Solution_list,Solution_list_data

Homotopy continuation corresponding to Algorithm 4.1.1 using Euler's Method, with specified maximum change:

In [8]:
#Sixth version of Homotopy Continuation:
def Homotopy_Continuation_6(F,M=100,specified_gamma=None,max_change=1000):
    #Define variables, and define x as variables that does not include t:
    variables=Var(F)
    t=variables[-1]
    x=variables[:-1]

    #Total degree start system G along with list of total degrees: 
    G,degrees=Total_degree_start_system(variables,F)
    
    #Set function in array for syntax:
    F=np.array(F)

    #Set stepsize h:
    h=1.0/M

    #Generate a random complex number gamma of modulus 1:
    if specified_gamma is None:
        c_number=complex(random.uniform(-1,1),random.uniform(-1,1))
        gamma=c_number/np.linalg.norm(c_number)
    else:
        gamma=specified_gamma

    #This defines our functions generated by the Davidenko equations
    H=t*F+gamma*(1-t)*G
    dtH = sp.lambdify(variables,sp.Matrix([H]).diff(t),'numpy')
    dxH = sp.lambdify(variables,sp.Matrix([H]).jacobian([x]),'numpy')
    H_eval=sp.lambdify(variables,H,'numpy')
    def euler_method(x_euler,t_euler,h):
        return x_euler-h * (np.linalg.inv(dxH(*(x_euler[:,0]),t_euler)) @ np.transpose(dtH(*(x_euler[:,0]),t_euler))) 
    def newton_method(x_newton, t_newton, tol,max_iterations):
        k=0
        H_newton=np.array(H_eval(*np.append((x_newton[:,0]),[t_newton]))).reshape(-1,1) #Initial value of H
        while any(np.linalg.norm(a) > tol for a in H_newton) and k<max_iterations: #While not convergent
            x_newton-= (np.linalg.inv(dxH(*x_newton[:,0], t_newton))@H_newton) #Update solutions
            H_newton=np.array(H_eval(*np.append((x_newton[:,0]),[t_newton]))).reshape(-1,1) #Update H
            k+=1
        if k == max_iterations:
            convergence=False
        else:
            convergence=True
        return x_newton, convergence,k
    def path_solution(x0_single, max_change, stepsize=1.0/M):
        plot_points=[x0_single]
        h=stepsize
        stepsize_end=stepsize/2
        t_j=0
        steps=0
        newton_iterations=0
        consecutive_convergences = 0    
        while t_j+h<1:  # Iterate through each step
            x0_prev=np.copy(x0_single)        
            # Predictor step (Euler's method)        
            x_pc = euler_method(x0_single, t_j,h)
            t_j+=h
            # Corrector step (Newton's method)
            if t_j>0.9:
                x0_single, convergence,k = newton_method(x_pc, t_j,tol=1e-9,max_iterations=3)
                stepsize=stepsize_end
                newton_iterations+=k            
            else:
                x0_single, convergence,k = newton_method(x_pc, t_j,tol=1e-6,max_iterations=1)
                newton_iterations+=k
            plot_points.append(x0_single)          
            if np.linalg.norm(x0_single-x0_prev) > max_change*h: #How do we deal with paths going to infinity
                return "Path is going to infinity", plot_points, steps, newton_iterations
            if convergence is True:
                consecutive_convergences += 1
                if consecutive_convergences >= 2:
                    h=2*stepsize  # Double the step size after 2 consecutive convergences
                else:
                    h=stepsize   
            else:
                h=stepsize/2  # Halve the step size if not converged
                consecutive_convergences = 0  # Reset counter
            steps+=1        
        if t_j+h>=1:
            h=1-t_j
            x0_prev=np.copy(x0_single)
            x_pc = euler_method(x0_single, t_j,h)
            x0_single = newton_method(x_pc, 1,tol=1e-12,max_iterations=10)[0]
            plot_points.append(x0_single)
            steps+=1
            newton_iterations+=k
            if np.linalg.norm(x0_single-x0_prev) > max_change*h: #How do we deal with paths going to infinity?
                return "Path is going to infinity", plot_points, steps, newton_iterations
        return x0_single, plot_points, steps, newton_iterations

    #Finding roots of G:
    roots_initial=[np.exp(2j * np.pi * np.arange(d) / d) for d in degrees]
    x0=[]
    #This gives us all combinations of solutions to our initial problem:
    for root0 in roots_initial[0]:
        x0.append([root0])
    for roots in roots_initial[1:]:
        np.ndarray.tolist(roots)
        x0=[a+[b] for a in x0 for b in roots]
    x_new=[]
    plot_x=[]
    steps_list=[]
    NI_list=[]
    for x0_sol in x0:
        x0_sol,plot_sol,steps, newton_iterations=path_solution(np.array(x0_sol).reshape(-1,1),max_change)
        x_new.append(x0_sol)
        plot_x.append(plot_sol)
        steps_list.append(steps)
        NI_list.append(newton_iterations)

    #Return solutions, steps and number of newton iterations:
    Solution_list=[]
    for i in range(len(x_new)):
        Solution_list.append(f"Solution {i+1}: {x_new[i]}")               
    return Solution_list

Homotopy continuation corresponding to Algorithm 4.1.1 using Runge-Kutta's Method, with specified starting system:

In [9]:
#Seventh version of Homotopy Continuation
def Homotopy_Continuation_7(F,G,degrees,M=100,specified_gamma=None): 
    #Define variables, and define x as variables that does not include t:
    variables=Var(F)
    t=variables[-1]
    x=variables[:-1]

    #Set function in array for syntax:
    F=np.array(F)
    G=np.array(G)
    #Set stepsize h:
    h=1.0/M

    #Generate a random complex number gamma of modulus 1:
    if specified_gamma is None:
        c_number=complex(random.uniform(-1,1),random.uniform(-1,1))
        gamma=c_number/np.linalg.norm(c_number)
    else:
        gamma=specified_gamma

    #This defines our functions generated by the Davidenko equations
    H=t*F+gamma*(1-t)*G
    dtH = sp.lambdify(variables,sp.Matrix([H]).diff(t),'numpy')
    dxH = sp.lambdify(variables,sp.Matrix([H]).jacobian([x]),'numpy')
    H_eval=sp.lambdify(variables,H,'numpy')
    def scipy_delta_x(x,t):
        return -(np.linalg.inv(dxH(*(x[:,0]),t)) @ np.transpose(dtH(*(x[:,0]),t))) 
    def runge_kutta(x_rk,t_rk,h):
        k1=scipy_delta_x(x_rk,t_rk)
        k2=scipy_delta_x(x_rk+(h/2)*k1,t_rk+(h/2))
        k3=scipy_delta_x(x_rk+(h/2)*k2,t_rk+(h/2))
        k4=scipy_delta_x(x_rk+h*k3,t_rk+h)
        return x_rk+(h/6)*(np.array(k1).reshape(-1,1)+2*np.array(k2).reshape(-1,1)+2*np.array(k3).reshape(-1,1)+np.array(k4).reshape(-1,1))
    def newton_method(x_newton, t_newton, tol=1e-6,max_iterations=3):
        k=0
        H_newton=np.array(H_eval(*np.append((x_newton[:,0]),[t_newton]))).reshape(-1,1) #Initial value of H
        while any(np.linalg.norm(a) > tol for a in H_newton) and k<max_iterations: #While not convergent
            x_newton-= (np.linalg.inv(dxH(*x_newton[:,0], t_newton))@H_newton) #Update solutions
            H_newton=np.array(H_eval(*np.append((x_newton[:,0]),[t_newton]))).reshape(-1,1) #Update H
            k+=1
        if k == max_iterations:
            convergence=False
        else:
            convergence=True
        return x_newton, convergence,k
    def path_solution(x0_single, stepsize=1.0/M, max_change=1000):
        plot_points=[x0_single]
        h=stepsize
        stepsize_end=stepsize/2
        t_j=0
        steps=0
        newton_iterations=0
        consecutive_convergences = 0    
        while t_j+h<1:  # Iterate through each step
            x0_prev=np.copy(x0_single)        
            # Predictor step (Euler's method)        
            x_pc = runge_kutta(x0_single, t_j,h)
            t_j+=h
            steps+=1 
            # Corrector step (Newton's method)
            if t_j>0.9:
                x0_single, convergence,k = newton_method(x_pc, t_j,tol=1e-9,max_iterations=3)
                stepsize=stepsize_end
                newton_iterations+=k            
            else:
                x0_single, convergence,k = newton_method(x_pc, t_j,tol=1e-6,max_iterations=1)
                newton_iterations+=k
            plot_points.append(x0_single)
            if np.linalg.norm(x0_single-x0_prev) > max_change*h: #How do we deal with paths going to infinity?
                return "Path is going to infinity", plot_points,steps,newton_iterations    
            if convergence is True:
                consecutive_convergences += 1
                if consecutive_convergences >= 2:
                    h=2*stepsize  # Double the step size after 2 consecutive convergences
                else:
                    h=stepsize   
            else:
                h=stepsize/2  # Halve the step size if not converged
                consecutive_convergences = 0  # Reset counter        
        if t_j+h>=1:
            h=1-t_j
            x0_prev=np.copy(x0_single)
            x_pc = runge_kutta(x0_single, t_j,h)
            x0_single, convergence,k = newton_method(x_pc, 1,tol=1e-12,max_iterations=10)
            plot_points.append(x0_single)
            steps+=1
            newton_iterations+=k
            if np.linalg.norm(x0_single-x0_prev) > max_change*h: #How we deal with paths going to infinity
                return "Path is going to infinity", plot_points,steps,newton_iterations
        return x0_single, plot_points, steps, newton_iterations

    #Finding roots of G:
    roots_initial=[np.exp(2j * np.pi * np.arange(d) / d) for d in degrees]
    x0=[]
    #This gives us all combinations of solutions to our initial problem:
    for root0 in roots_initial[0]:
        x0.append([root0])
    for roots in roots_initial[1:]:
        np.ndarray.tolist(roots)
        x0=[a+[b] for a in x0 for b in roots]
    x_new=[]
    plot_x=[]
    steps_list=[]
    NI_list=[]
    for x0_sol in x0:
        x0_sol,plot_sol,steps, newton_iterations=path_solution(np.array(x0_sol).reshape(-1,1))
        x_new.append(x0_sol)
        plot_x.append(plot_sol)
        steps_list.append(steps)
        NI_list.append(newton_iterations)
    
    #Plot solution paths:
    fig, ax = plt.subplots()
    for xi_plotlist in plot_x:
        # Extract real and imaginary parts of the solutions
        real_parts = [x.real for x in np.squeeze(xi_plotlist)]
        imag_parts = [x.imag for x in np.squeeze(xi_plotlist)]
        # Plot the path for the current root
        color = next(ax._get_lines.prop_cycler)['color'] #Ensures end points has same color as path
        ax.plot(real_parts, imag_parts, marker=',', linestyle='-',color=color)
        ax.plot(real_parts[0], imag_parts[0], marker='s', linestyle='-',color=color) #Path starts with a square.
        ax.plot(real_parts[-1], imag_parts[-1], marker='x', linestyle='-',color=color) #Path ends in an asterix.
    ax.set_xlabel('Real Part')
    ax.set_ylabel('Imaginary Part')
    ax.set_title(f'Solution Paths for the Homotopy h: {H}')
    plt.grid(True)
    plt.show() 
    
    #Return solutions, steps and number of newton iterations:
    Solution_list=[]
    for i in range(len(x_new)):
        Solution_list.append(f"Solution {i+1}: {x_new[i]}")
    return Solution_list

Homotopy continuation corresponding to Algorithm 4.1.1 using Runge-Kutta with projective transformation as specified input:

In [10]:
#Eighth version of Homotopy Continuation:
def Homotopy_Continuation_8_proj(F,G,degrees,projective_transformation,M=100):
    #Define variables, and define x as variables that does not include t:
    variables=Var(F)
             
    #Homogenize function and set in array for syntax:
    F=Homogenize_with_respect_to_total_degree_of_G(variables, F, G)
    G=Homogenize_function(variables, G)
    F=np.array(F)
    G=np.array(G)

    #Coefficients for specified projective transformation:
    a=projective_transformation[0] #Multiply by x0
    b=projective_transformation[1] #Multiply by x1

    #Set stepsize h:
    h=1.0/M

    #Include x0 as a variable and set x for variables that does not include t:
    x0=sp.symbols('x0')
    variables=(x0,*variables)
    t=variables[-1]
    x=variables[:-1]

    #This defines our functions generated by the Davidenko equations
    H=t*F+(1-t)*G
    H=np.append(H,a*x[0]+b*x[1]-1)
    dtH = sp.lambdify(variables,sp.Matrix([H]).diff(t),'numpy')
    dxH = sp.lambdify(variables,sp.Matrix([H]).jacobian([x]),'numpy')
    H_eval=sp.lambdify(variables,H,'numpy')
    def scipy_delta_x(x,t):
        return -(np.linalg.inv(dxH(*(x[:,0]),t)) @ np.transpose(dtH(*(x[:,0]),t))) 
    def runge_kutta(x_rk,t_rk,h):
        k1=scipy_delta_x(x_rk,t_rk)
        k2=scipy_delta_x(x_rk+(h/2)*k1,t_rk+(h/2))
        k3=scipy_delta_x(x_rk+(h/2)*k2,t_rk+(h/2))
        k4=scipy_delta_x(x_rk+h*k3,t_rk+h)
        return x_rk+(h/6)*(np.array(k1).reshape(-1,1)+2*np.array(k2).reshape(-1,1)+2*np.array(k3).reshape(-1,1)+np.array(k4).reshape(-1,1))
    def newton_method(x_newton, t_newton, tol=1e-6,max_iterations=3):
        k=0
        H_newton=np.array(H_eval(*np.append((x_newton[:,0]),[t_newton]))).reshape(-1,1) #Initial value of H
        while any(np.linalg.norm(a) > tol for a in H_newton) and k<max_iterations: #While not convergent
            x_newton-= (np.linalg.inv(dxH(*x_newton[:,0], t_newton))@H_newton) #Update solutions
            H_newton=np.array(H_eval(*np.append((x_newton[:,0]),[t_newton]))).reshape(-1,1) #Update H
            k+=1
        if k == max_iterations:
            convergence=False
        else:
            convergence=True
        return x_newton, convergence
    def path_solution(x0_single, stepsize=1.0/M, min_frac=1e-12):
        plot_points=[x0_single[1]]
        h=stepsize
        stepsize_end=stepsize/2
        t_j=0
        consecutive_convergences = 0    
        while t_j+h<1:  # Iterate through each step
            # Predictor step (Euler's method)        
            x_pc = runge_kutta(x0_single, t_j,h)
            t_j+=h     
            # Corrector step (Newton's method)   
            if t_j>0.9:
                x0_single, convergence = newton_method(x_pc, t_j,tol=1e-9,max_iterations=3)
                stepsize=stepsize_end
            else:
                x0_single, convergence = newton_method(x_pc, t_j,tol=1e-6,max_iterations=1)                
            plot_points.append(x0_single[1])
            if convergence is True:
                consecutive_convergences += 1
                if consecutive_convergences >= 2:
                    h=2*stepsize  # Double the step size after 2 consecutive convergences
                else:
                    h=stepsize        
            else:
                h=stepsize/2  # Halve the step size if not converged
                consecutive_convergences = 0  # Reset counter
        if t_j+h>=1:
            h=1-t_j
            x_pc = runge_kutta(x0_single, t_j,h)
            x0_single = newton_method(x_pc, 1,tol=1e-12,max_iterations=10)[0]
            plot_points.append(x0_single[1])
            #We only check if paths are going to infinity at the end, since the projective transformation keeps the paths from going to infinity.
            for xj in x0_single[1:]: #Iterate over all coordinates of x.
                if abs(x0_single[0])/(abs(xj))<min_frac: #Checks if the fraction of modulus of x0 and the other coordinates is very small.
                    return f"Path is going to point at infinity {x0_single}", plot_points #If the fraction is sufficiently small, it means the path goes to the point at infinity
        return x0_single, plot_points
    #Finding roots of G:
    roots_initial=[np.exp(2j * np.pi * np.arange(d) / d) for d in degrees]
    x_start=[]
    x_new=[]
    #This gives us all combinations of solutions to our initial problem:
    for root0 in roots_initial[0]:
        x_start.append([root0])
    for roots in roots_initial[1:]:
        np.ndarray.tolist(roots)
        x_start=[a+[b] for a in x_start for b in roots]
    x_start_proj=[]
    for i in x_start:
        projroot=np.append([1/(a+b*i[0])],i[0]/(a+b*i[0]))
        x_start_proj.append(projroot)
    x_start=x_start_proj
    plot_x=[]
    for x0_sol in x_start:
        x0_sol,plot_sol=path_solution(np.array(x0_sol).reshape(-1,1))   
        x_new.append(x0_sol)
        plot_x.append(plot_sol)
        
    #Plot solution paths:
    fig, ax = plt.subplots()
    for xi_plotlist in plot_x:
        # Extract real and imaginary parts of the solutions
        real_parts = [x.real for x in np.squeeze(xi_plotlist)]
        imag_parts = [x.imag for x in np.squeeze(xi_plotlist)]
        # Plot the path for the current root
        color = next(ax._get_lines.prop_cycler)['color'] #Ensures end points has same color as path
        ax.plot(real_parts, imag_parts, marker=',', linestyle='-',color=color)
        ax.plot(real_parts[0], imag_parts[0], marker='s', linestyle='-',color=color)
        ax.plot(real_parts[-1], imag_parts[-1], marker='x', linestyle='-',color=color)
    ax.set_xlabel('Real Part')     
    ax.set_ylabel('Imaginary Part')
    ax.set_title(f'Projective Transformation of Solution Paths for x1 of the polynomial: F={F}')
    plt.grid(True)
    plt.show()

    #Return solutions, steps and number of newton iterations:
    Solution_list_projective=[]
    Solution_list_affine=[]
    for i in range(len(x_new)):
        Solution_list_projective.append(f"Projective Solution {i+1}: {x_new[i]}")
        if isinstance(x_new[i],str):
            Solution_list_affine.append(f"Affine Solution {i+1}: Path is going to infinity")
        else:
            Solution_list_affine.append(f"Affine Solution {i+1}: {x_new[i][1]/x_new[i][0]}")                       
    return Solution_list_affine, Solution_list_projective

Algorithm 5.2.1 (a): General Homotopy Continuation using Algorithm 4.1.1 with Runge-Kutta and projective transformation:

In [11]:
#Ninth version of Homotopy Continuation:
def Homotopy_Continuation_A(F,M=1000):
    #Define variables, and define x as variables that does not include t:
    variables=Var(F)
   
    #Homogeneous total degree start system G along with list of total degrees: 
    G,degrees=Homogenized_total_degree_start_system(variables,F)
    
    #Homogenize function and set in array for syntax:
    F=Homogenize_function(variables, F)
    F=np.array(F)
    
    #Set stepsize h:
    h=1.0/M

    #Include x0 as a variable and set x for variables that does not include t:
    x0=sp.symbols('x0')
    variables=(x0,*variables)
    t=variables[-1]
    x=variables[:-1]

    #Projective transformation:
    proj_coeff_list=[]
    for i in range(len(x)):
        c_number=complex(random.uniform(-1,1),random.uniform(-1,1)) #Create a coefficient for every variable including x0
        proj_coeff_list.append(c_number/np.linalg.norm(c_number)) #Use coefficient with norm one.
    projective_transformation=0
    for i in range(len(x)):
        projective_transformation+=proj_coeff_list[i]*x[i] #Equaition for projective transformation.

    #Functions for the homotopy:
    H=projective_transformation-1 #Defines projective transformation as part of H.
    H=np.append(H,t*F+(1-t)*G) #Appends rest of H.
    dtH = sp.lambdify(variables,sp.Matrix([H]).diff(t),'numpy')
    dxH = sp.lambdify(variables,sp.Matrix([H]).jacobian([x]),'numpy')
    H_eval=sp.lambdify(variables,H,'numpy')
    def scipy_delta_x(x,t):
        return -(np.linalg.inv(dxH(*(x[:,0]),t)) @ np.transpose(dtH(*(x[:,0]),t))) 
    def runge_kutta(x_rk,t_rk,h):
        k1=scipy_delta_x(x_rk,t_rk)
        k2=scipy_delta_x(x_rk+(h/2)*k1,t_rk+(h/2))
        k3=scipy_delta_x(x_rk+(h/2)*k2,t_rk+(h/2))
        k4=scipy_delta_x(x_rk+h*k3,t_rk+h)
        return x_rk+(h/6)*(np.array(k1).reshape(-1,1)+2*np.array(k2).reshape(-1,1)+2*np.array(k3).reshape(-1,1)+np.array(k4).reshape(-1,1))
    def newton_method(x_newton, t_newton, tol=1e-6,max_iterations=3):
        k=0
        H_newton=np.array(H_eval(*np.append((x_newton[:,0]),[t_newton]))).reshape(-1,1) #Initial value of H
        while any(np.linalg.norm(a) > tol for a in H_newton) and k<max_iterations: #While not convergent
            x_newton-= (np.linalg.inv(dxH(*x_newton[:,0], t_newton))@H_newton) #Update solutions
            H_newton=np.array(H_eval(*np.append((x_newton[:,0]),[t_newton]))).reshape(-1,1) #Update H
            k+=1
        if k == max_iterations:
            convergence=False
        else:
            convergence=True
        return x_newton, convergence
    def path_solution(x0_single, stepsize=1.0/M, min_frac=1e-12):
        plot_points=[x0_single[1]]
        h=stepsize
        stepsize_end=stepsize/2
        t_j=0
        consecutive_convergences = 0    
        while t_j+h<1:  # Iterate through each step
            # Predictor step (Euler's method)        
            x_pc = runge_kutta(x0_single, t_j,h)
            t_j+=h     
            # Corrector step (Newton's method)   
            if t_j>0.9:
                x0_single, convergence = newton_method(x_pc, t_j,tol=1e-9,max_iterations=3)
                stepsize=stepsize_end
            else:
                x0_single, convergence = newton_method(x_pc, t_j,tol=1e-6,max_iterations=1)                
            plot_points.append(x0_single[1])
            if convergence is True:
                consecutive_convergences += 1
                if consecutive_convergences >= 2:
                    h=2*stepsize  # Double the step size after 2 consecutive convergences
                else:
                    h=stepsize        
            else:
                h=stepsize/2  # Halve the step size if not converged
                consecutive_convergences = 0  # Reset counter
        if t_j+h>=1:
            h=1-t_j
            x_pc = runge_kutta(x0_single, t_j,h)
            x0_single = newton_method(x_pc, 1,tol=1e-12,max_iterations=10)[0]
            plot_points.append(x0_single[1])
        for xj in x0_single[1:]: #Check if path goes to point at infinity
            if abs(x0_single[0])/(abs(xj))<min_frac:
                return f"Path is going to point at infinity {x0_single}", plot_points
        return x0_single, plot_points
    #Finding roots of G:
    roots_initial=[np.exp(2j * np.pi * np.arange(d) / d) for d in degrees]
    x_start=[]
    x_new=[]
    #This gives us all combinations of solutions to our initial problem:
    for root0 in roots_initial[0]:
        x_start.append([root0])
    for roots in roots_initial[1:]:
        np.ndarray.tolist(roots)
        x_start=[a+[b] for a in x_start for b in roots]
    #Projective transformation of initial roots.
    x_start_proj=[]
    denominatorlist=[]
    for i in range(len(x_start)): #Iterate over iniital roots.
        denominator=proj_coeff_list[0] #First coefficient of projective transformation. 
        for j in range(len(x_start[i])): #iterate over coordinates in each solution.
            xi_den=proj_coeff_list[j+1]*x_start[i][j] #Multiply by corresponding coefficient of projective transformation for x1...xn, since we assume x0=1.
            denominator+=xi_den #Sum of the projective transformation at each coordinate.
        denominatorlist.append(denominator) #Collect projective transformation for each initial solution.
    for i in range(len(x_start)): #Iterate over initial roots again.
        projroot=np.array(1/denominatorlist[i]) #Divide x0=1 by the projective transformation.
        for j in range(len(x_start[i])): #Iterate over coordinates in each solution.
            xi=x_start[i][j]/denominatorlist[i] #Divide each coordinate by the projective transformation
            projroot=np.append(projroot,xi) #Initial solution at given coordinates
        x_start_proj.append(projroot) #Collect all projective transformations of initial solutions.
    x_start=x_start_proj

    plot_x=[]
    for x0_sol in x_start:
        x0_sol,plot_sol=path_solution(np.array(x0_sol).reshape(-1,1))   
        x_new.append(x0_sol)
        plot_x.append(plot_sol)

    #Return solutions, steps and number of newton iterations:
    Solutions=[]
    Solution_list_projective=[]
    Solution_list_affine=[]
    for i in range(len(x_new)):
        Solution_list_projective.append(f"Projective Solution {i+1}: {x_new[i]}")
        if isinstance(x_new[i],str):
            Solution_list_affine.append(f"Affine Solution {i+1}: Path is going to infinity")
        else:
            Solution_list_affine.append(f"Affine Solution {i+1}: {x_new[i][1:]/x_new[i][0]}")   
            Solutions.append(x_new[i][1:]/x_new[i][0])                            
    return Solutions,Solution_list_affine, Solution_list_projective

Algorithm 5.2.1 (b): General Homotopy Continuation using Algorithm 4.2.1 and projective transformation:

In [12]:
#Tenth version of Homotopy Continuation:
def Homotopy_Continuation_B(F,M=1000):
    #Define variables, and define x as variables that does not include t:
    variables=Var(F)

    #Homogeneous total degree start system G along with list of total degrees: 
    G,degrees=Homogenized_total_degree_start_system(variables,F)
    
    #Homogenize function and set in array for syntax:
    F=Homogenize_function(variables, F)
    F=np.array(F)
    
    #Set stepsize h:
    h=1.0/M

    #Include x0 as a variable and set x for variables that does not include t:
    x0=sp.symbols('x0')
    variables=(x0,*variables)
    t=variables[-1]
    x=variables[:-1]

    #Projective transformation:
    proj_coeff_list=[]
    for i in range(len(x)):
        c_number=complex(random.uniform(-1,1),random.uniform(-1,1)) #Create a coefficient for every variable including x0
        proj_coeff_list.append(c_number/np.linalg.norm(c_number)) #Use coefficient with norm one.
    projective_transformation=0
    for i in range(len(x)):
        projective_transformation+=proj_coeff_list[i]*x[i] #Equaition for projective transformation.

    #Functions for the homotopy:
    H=projective_transformation-1 #Defines projective transformation as part of H.
    H=np.append(H,t*F+(1-t)*G) #Appends rest of H.
    dtH = sp.lambdify(variables,sp.Matrix([H]).diff(t),'numpy')
    dxH = sp.lambdify(variables,sp.Matrix([H]).jacobian([x]),'numpy')
    H_eval=sp.lambdify(variables,H,'numpy')
    def scipy_delta_x(x,t):
        return -(np.linalg.inv(dxH(*(x[:,0]),t)) @ np.transpose(dtH(*(x[:,0]),t))) 
    def euler_method(x_euler,t_euler,h):
        return x_euler+h * scipy_delta_x(x_euler,t_euler)
    def runge_kutta(x_rk,t_rk,h):
        k1=scipy_delta_x(x_rk,t_rk)
        k2=scipy_delta_x(x_rk+(h/2)*k1,t_rk+(h/2))
        k3=scipy_delta_x(x_rk+(h/2)*k2,t_rk+(h/2))
        k4=scipy_delta_x(x_rk+h*k3,t_rk+h)
        return x_rk+(h/6)*(np.array(k1).reshape(-1,1)+2*np.array(k2).reshape(-1,1)+2*np.array(k3).reshape(-1,1)+np.array(k4).reshape(-1,1))
    def adams_bashfort_method(x_ab0,x_ab1,x_ab2,x_ab3,t_ab0,t_ab1,t_ab2,t_ab3,h):
        x_ab0_diff=scipy_delta_x(x_ab0,t_ab0)
        x_ab1_diff=scipy_delta_x(x_ab1,t_ab1)
        x_ab2_diff=scipy_delta_x(x_ab2,t_ab2)
        x_ab3_diff=scipy_delta_x(x_ab3,t_ab3)
        return x_ab3+h/24*(55*x_ab3_diff-59*x_ab2_diff+37*x_ab1_diff-9*x_ab0_diff)
    def adams_moulton_method(x_ab0,x_ab1,x_ab2,x_ab3,t_ab0,t_ab1,t_ab2,t_ab3,h):
        x_ab0_diff=scipy_delta_x(x_ab0,t_ab0)
        x_ab1_diff=scipy_delta_x(x_ab1,t_ab1)
        x_ab2_diff=scipy_delta_x(x_ab2,t_ab2)
        x_ab3_diff=scipy_delta_x(x_ab3,t_ab3)
        return x_ab2+(h/24)*(9*x_ab3_diff+19*x_ab2_diff-5*x_ab1_diff+x_ab0_diff)
    def abam_method(x_ab0,x_ab1,x_ab2,x_ab3,t_ab0,t_ab1,t_ab2,t_ab3,h):
        x_forward=adams_bashfort_method(x_ab0,x_ab1,x_ab2,x_ab3,t_ab0,t_ab1,t_ab2,t_ab3,h)
        x_backward=adams_moulton_method(x_ab1,x_ab2,x_ab3,x_forward,t_ab1,t_ab2,t_ab3,t_ab3+h,h)
        return x_backward
    def newton_method(x_newton, t_newton, tol=1e-6,max_iterations=3):
        k=0
        H_newton=np.array(H_eval(*np.append((x_newton[:,0]),[t_newton]))).reshape(-1,1) #Initial value of H
        while any(np.linalg.norm(a) > tol for a in H_newton) and k<max_iterations: #While not convergent
            x_newton-= (np.linalg.inv(dxH(*x_newton[:,0], t_newton))@H_newton) #Update solutions
            H_newton=np.array(H_eval(*np.append((x_newton[:,0]),[t_newton]))).reshape(-1,1) #Update H
            k+=1
        if k == max_iterations:
            convergence=False
        else:
            convergence=True
        return x_newton, convergence
    def path_solution(x0_single, stepsize=1.0/M, min_frac=1e-12):
        plot_points=[x0_single[1]]
        x_list=[x0_single]
        h=stepsize
        t_j=0
        for j in range(0,M):  # Iterate through each step
            t_j = j * h
            max_eig,min_eig=max(abs(np.linalg.eigvals(dxH(*scipy_delta_x(x0_single,t_j)[:,0],t_j)))),min(abs(np.linalg.eigvals(dxH(*scipy_delta_x(x0_single,t_j)[:,0],t_j)))) #Largest and smallest eignevalues of Jacobian of change of x with respect to absolute value
            if j<3:
                x_pc=runge_kutta(x0_single,t_j,h)
            else:
                if max_eig<10 and max_eig/min_eig<10: #Checks if the difference is relatively large
                    x_pc=euler_method(x0_single,t_j,h)
                else:
                    x_pc=abam_method(x_list[j-3],x_list[j-2],x_list[j-1],x_list[j],t_j-3*h,t_j-2*h,t_j-h,t_j,h) #Use predictor for stiff problems
            
            if 1>t_j>0.9:
                x0_single=newton_method(x_pc, t_j+h,tol=1e-9,max_iterations=3)[0]
            elif j==(M-1):
                x0_single = newton_method(x_pc,1,tol=1e-12,max_iterations=10)[0]
            else:
                x0_single=newton_method(x_pc, t_j+h,tol=1e-6,max_iterations=1)[0]
            x_list.append(x0_single)        
            plot_points.append(x0_single[1])
        plot_points.append(x0_single[1])
        for xj in x0_single[1:]:
            if abs(x0_single[0])/(abs(xj))<min_frac:
                return f"Path is going to point at infinity {x0_single}", plot_points
        return x0_single, plot_points
    #Finding roots of G:
    roots_initial=[np.exp(2j * np.pi * np.arange(d) / d) for d in degrees]
    x_start=[]
    x_new=[]
    #This gives us all combinations of solutions to our initial problem:
    for root0 in roots_initial[0]:
        x_start.append([root0])
    for roots in roots_initial[1:]:
        np.ndarray.tolist(roots)
        x_start=[a+[b] for a in x_start for b in roots]
    x_start_proj=[]
    denominatorlist=[]
    for i in range(len(x_start)): #Iterate over iniital roots.
        denominator=proj_coeff_list[0] #First coefficient of projective transformation. 
        for j in range(len(x_start[i])): #iterate over coordinates in each solution.
            xi_den=proj_coeff_list[j+1]*x_start[i][j] #Multiply by corresponding coefficient of projective transformation for x1...xn, since we assume x0=1.
            denominator+=xi_den #Sum of the projective transformation at each coordinate.
        denominatorlist.append(denominator) #Collect projective transformation for each initial solution.
    for i in range(len(x_start)): #Iterate over initial roots again.
        projroot=np.array(1/denominatorlist[i]) #Divide x0=1 by the projective transformation.
        for j in range(len(x_start[i])): #Iterate over coordinates in each solution.
            xi=x_start[i][j]/denominatorlist[i] #Divide each coordinate by the projective transformation
            projroot=np.append(projroot,xi) #Initial solution at given coordinates
        x_start_proj.append(projroot) #Collect all projective transformations of initial solutions.
    x_start=x_start_proj

    plot_x=[]
    for x0_sol in x_start:
        x0_sol,plot_sol=path_solution(np.array(x0_sol).reshape(-1,1))   
        x_new.append(x0_sol)
        plot_x.append(plot_sol)

    #Return solutions, steps and number of newton iterations:
    Solutions=[]
    Solution_list_projective=[]
    Solution_list_affine=[]
    for i in range(len(x_new)):
        Solution_list_projective.append(f"Projective Solution {i+1}: {x_new[i]}")
        if isinstance(x_new[i],str):
            Solution_list_affine.append(f"Affine Solution {i+1}: Path is going to infinity")
        else:
            Solution_list_affine.append(f"Affine Solution {i+1}: {x_new[i][1:]/x_new[i][0]}")
            Solutions.append(x_new[i][1:]/x_new[i][0])
    return Solutions, Solution_list_affine, Solution_list_projective

Coefficient-Paramter Homotopies:

In [13]:
#Coefficient-Paramter Homotopy Continuation:
def Homotopy_Continuation_CP(F0,F0_roots,F1,U_complement=1,M=1000,specified_gamma=None,U_min_frac=1e-12):
    #Define variables, and define x as variables that does not include t:
    variables=Var(F0)
    t=variables[-1]
    x=variables[:-1]
    #Homogenize function and set in array for syntax:
    F0=np.array(F0)
    F1=np.array(F1)
    
    #Set stepsize h:
    h=1.0/M

    #Generate a random complex number gamma of modulus 1:
    if specified_gamma is None:
        c_number=complex(random.uniform(-1,1),random.uniform(-1,1))
        gamma=c_number/np.linalg.norm(c_number)
    else:
        gamma=specified_gamma

    #Functions for the homotopy:
    U_complement_eval=sp.lambdify(x,U_complement,'numpy') #Evaluate equation defined by the Zariski open set U.
    H=t*F1+(1-t)*F0 #Appends rest of H.
    dtH = sp.lambdify(variables,sp.Matrix(H).diff(t),'numpy')
    dxH = sp.lambdify(variables,sp.Matrix(H).jacobian([x]),'numpy')
    H_eval=sp.lambdify(variables,H,'numpy')
    def scipy_delta_x(x,t):
        return -(np.linalg.inv(dxH(*(x[:,0]),t)) @ (dtH(*(x[:,0]),t)))
    def euler_method(x_euler,t_euler,h):
        return x_euler+h * scipy_delta_x(x_euler,t_euler)
    def runge_kutta(x_rk,t_rk,h):
        k1=scipy_delta_x(x_rk,t_rk)
        k2=scipy_delta_x(x_rk+(h/2)*k1,t_rk+(h/2))
        k3=scipy_delta_x(x_rk+(h/2)*k2,t_rk+(h/2))
        k4=scipy_delta_x(x_rk+h*k3,t_rk+h)
        return x_rk+(h/6)*(np.array(k1).reshape(-1,1)+2*np.array(k2).reshape(-1,1)+2*np.array(k3).reshape(-1,1)+np.array(k4).reshape(-1,1))
    def adams_bashfort_method(x_ab0,x_ab1,x_ab2,x_ab3,t_ab0,t_ab1,t_ab2,t_ab3,h):
        x_ab0_diff=scipy_delta_x(x_ab0,t_ab0)
        x_ab1_diff=scipy_delta_x(x_ab1,t_ab1)
        x_ab2_diff=scipy_delta_x(x_ab2,t_ab2)
        x_ab3_diff=scipy_delta_x(x_ab3,t_ab3)
        return x_ab3+h/24*(55*x_ab3_diff-59*x_ab2_diff+37*x_ab1_diff-9*x_ab0_diff)
    def adams_moulton_method(x_ab0,x_ab1,x_ab2,x_ab3,t_ab0,t_ab1,t_ab2,t_ab3,h):
        x_ab0_diff=scipy_delta_x(x_ab0,t_ab0)
        x_ab1_diff=scipy_delta_x(x_ab1,t_ab1)
        x_ab2_diff=scipy_delta_x(x_ab2,t_ab2)
        x_ab3_diff=scipy_delta_x(x_ab3,t_ab3)
        return x_ab2+(h/24)*(9*x_ab3_diff+19*x_ab2_diff-5*x_ab1_diff+x_ab0_diff)
    def abam_method(x_ab0,x_ab1,x_ab2,x_ab3,t_ab0,t_ab1,t_ab2,t_ab3,h):
        x_forward=adams_bashfort_method(x_ab0,x_ab1,x_ab2,x_ab3,t_ab0,t_ab1,t_ab2,t_ab3,h)
        x_backward=adams_moulton_method(x_ab1,x_ab2,x_ab3,x_forward,t_ab1,t_ab2,t_ab3,t_ab3+h,h)
        return x_backward
    def newton_method(x_newton, t_newton, tol=1e-6,max_iterations=3):
        k=0
        H_newton=np.array(H_eval(*np.append((x_newton[:,0]),[t_newton]))).reshape(-1,1) #Initial value of H
        while any(np.linalg.norm(a) > tol for a in H_newton) and k<max_iterations: #While not convergent
            x_newton-= (np.linalg.inv(dxH(*x_newton[:,0], t_newton))@H_newton) #Update solutions
            H_newton=np.array(H_eval(*np.append((x_newton[:,0]),[t_newton]))).reshape(-1,1) #Update H
            k+=1
        if k == max_iterations:
            convergence=False
        else:
            convergence=True
        return x_newton, convergence
    
    def gamma_path_solution(x0_single, stepsize=1.0/M): #U is defined as a Zariski open subset
        x_list=[x0_single]
        h=stepsize
        t_j=0
        for j in range(0,M):  # Iterate through each step
            t_j = j*gamma/(1+(gamma-1)*j)
            max_eig,min_eig=max(abs(np.linalg.eigvals(dxH(*scipy_delta_x(x0_single,t_j)[:,0],t_j)))),min(abs(np.linalg.eigvals(dxH(*scipy_delta_x(x0_single,t_j)[:,0],t_j)))) #Largest and smallest eignevalues of Jacobian of change of x with respect to absolute value
            if j<3:
                x_pc=runge_kutta(x0_single,t_j,h)
            else:
                if max_eig<10 and max_eig/min_eig<10: #Checks if the difference is relatively large
                    x_pc=euler_method(x0_single,t_j,h)
                else:
                    x_pc=abam_method(x_list[j-3],x_list[j-2],x_list[j-1],x_list[j],t_j-3*h,t_j-2*h,t_j-h,t_j,h) #Use predictor for stiff problems
            
            if 1>abs(t_j)>0.9:
                x0_single=newton_method(x_pc, t_j+h,tol=1e-9,max_iterations=3)[0]
            elif j==(M-1):
                x0_single = newton_method(x_pc,1,tol=1e-12,max_iterations=10)[0]
            else:
                x0_single=newton_method(x_pc, t_j+h,tol=1e-6,max_iterations=1)[0]
            x_list.append(x0_single)
        return x0_single #Return solutions that are in U.
    x_new_parameter=[]
    for x0_sol in F0_roots:
        x0_sol=gamma_path_solution(np.array(x0_sol).reshape(-1,1))
        U_x=U_complement_eval(*(x0_sol[:,0])) #Evaluate U at this solution
        if not isinstance(U_complement, tuple): #Check if U_complement is more than one polynomial and ensures we can iterate over the polynomials
            U_x = [U_x]
        for poly in U_x: #Check if each polynomial is very close to 0
            if abs(poly)>U_min_frac: #If one of the polynomial is not close to 0, it is not in the variety, meaning the solution lies in U.
                x_new_parameter.append(x0_sol)
                break
    #Return solutions for new parameters:                      
    return x_new_parameter

Certification:

In [15]:
#Determining the unit roundoff:
u=1.0
while (1.0+0.5*u)>1.0: #Half it until it cannot distinguish between the number and half the number.
    u=0.5*u
#Function that generates an interval that is compatible with regular operations.
def interval(re,im):
    return iv.mpc(f'{re}',f'{im}')
#We can just use +,-,*,/ for this package.
#Function that performs scalar multiplication on a vector of intervals.
def scalar_multiplication(scalar,vector):
    if isinstance(vector,np.ndarray):
        new_vector=np.zeros(len(vector),dtype=object)
        for i in range(len(vector)):
            new_vector[i]=scalar*vector[i] #We multiply every element in the vector by the scalar
    else:
        new_vector=scalar*vector
    return new_vector
#Function that performs products of matrices of intervals and vectors of intervals.
def matrix_vector_prod(matrix,vector):
    if isinstance(vector,np.ndarray):        
        new_vector=np.zeros(len(vector),dtype=object)
        if len(vector)==1:
            return matrix*vector
        for i in range(len(vector)):
            new_vector+=scalar_multiplication(vector[i],matrix[:,i]) #We multply every interval in the vector by each column in the matrix and sum it.
    else:
        new_vector=matrix*vector
    return new_vector
#Function that creates an identity matrix, which is compatible with the interval arithmetics, meaning it contains the neutral elements [1,1]+i[0,0] in the diagonal.
def Identity_matrix(dimension):
    Identity=np.zeros((dimension,dimension),dtype=object)
    for i in range(dimension):
        for j in range(dimension):
            if j==i:
                Identity[i,j]=interval([1,1],[0,0])    
            else:
                Identity[i,j]=interval([0,0],[0,0])
    return Identity
#Function defining the infinity norm of interval.
def max_norm_interval(Interval):
    IR,II=Interval.real,Interval.imag
    return np.sqrt(max(float(IR.a)**2,float(IR.b)**2)+max(float(II.a)**2,float(II.b)**2)) #The end point of a real interval is equal to the maximum after taking the square root
#Function defining the infinity norm of a matrix.
def infinity_matrix_norm(A):
    if isinstance(A,np.ndarray):
        sum_row_norms_list=[]
        for i in range(len(A)):
            sum_row_norms=0
            for j in range(len(A)): #Sum over max norm of intervals in each entry in a column.
                sum_row_norms+=max_norm_interval(A[i][j])
            sum_row_norms_list.append(sum_row_norms)
        max_norm_matrix=max(sum_row_norms_list) #The max norm of the matrix is the maximum of these sums.
    else:
        max_norm_matrix=max_norm_interval(A)
    return max_norm_matrix
#Function that defines the Krawczyk operator.
def Krawczyk(F,I,A,x,JF_square):
    accuracylist=[]
    Strong_interval,Real_solution,accuracy=0,0,0
    if isinstance(x,np.ndarray):
        Identity=Identity_matrix(len(x)) #Create n-dimensional identity matrix
        K_1=x-matrix_vector_prod(A,F(x)) #Construct the first term of the Krawczyk operator.
        K_2=Identity-A@JF_square(I) #Construct part of the second term of the Krawczyk operator.
        Krawczyk=K_1+matrix_vector_prod((K_2),(I-x)) #Construct the Krawczyk operator using these terms.
        Strong_interval_norm=infinity_matrix_norm(K_2) #Norm condition of interval to be strong.
        if all(Krawczyk[i] in I[i] for i in range(len(I))): #Tests if Krawczyk operator is in the interval
            Approx_int=True
        else:
            Approx_int=False
        if Strong_interval_norm<1 and Approx_int==True: #Tests if interval is strong
            Strong_interval=I
            Strong=True
        else:
            Strong=False
        if Strong==True: #If we have a strong interval approximate 0:
            Kraw_conjugate=np.zeros(len(Krawczyk),dtype=object) #Constructs the complex conjugate of the Krawczyk operator
            for i in range(len(Krawczyk)):
                Kraw_conjugate[i]=interval(Krawczyk[i].real,-(Krawczyk[i].imag))
                accuracylist.append(float((Krawczyk[i].real).delta)) #Keep lengths of intervals real and imaginary parts
                accuracylist.append(float((Krawczyk[i].imag).delta))           
            if all(Kraw_conjugate[i] in Strong_interval[i] for i in range(len(Strong_interval))): #Checks if solution is real
                Real_solution=True
            elif any(interval([0,0],[0,0]).imag not in Strong_interval[i].imag for i in range(len(Strong_interval))): #Checks if solution is non-real
                Real_solution=False
            accuracy=max(accuracylist)**(0.5) #Set to largest possible numerical inaccuracy per coordinate, which is length between the corners of the interval.
        else:
            Real_solution="Real or Complex not determined"
    else: # Do the same in one-dimensional case.
        Identity=Identity_matrix(1)
        F_eval,JF_square_eval=F(x),JF_square(I)
        K_1=x-A*F_eval
        K_2=(Identity-A*JF_square_eval)[0][0]
        Krawczyk=K_1+K_2*(I-x)
        Strong_interval_norm=infinity_matrix_norm(K_2)
        if Krawczyk[i] in I[i]:
            Approx_int=True
        else:
            Approx_int=False
        if Strong_interval_norm<1  and Approx_int==True:
            Strong_interval=I
            Strong=True
        else:
            Strong=False
        if Strong==True:
            Kraw_conjugate=interval(Krawczyk.real,-(Krawczyk.imag))
            accuracy=max(float((Krawczyk[i].real).delta),float((Krawczyk[i].imag).delta))**(0.5)
            if Kraw_conjugate in Strong_interval:
                Real_solution=True
            if interval([0,0],[0,0]).imag not in Strong_interval.imag:
                Real_solution=False
        else:
            Real_solution="Real or Complex not determined"
    return Approx_int, Strong, Real_solution, accuracy

#Function that certifies the solutions of a function:
def certify(F, sol):
    #Define variables, and define x as variables that does not include t:
    variables=Var(F)
    t=variables[-1]
    x=variables[:-1]
    def F_int(I): #Make the polynomial system F compatible with intervals.
        F_new=sp.lambdify(x,[F],'numpy')
        if isinstance(I,np.ndarray):
            F_eval=np.array(F_new(*I))[0]
        else:
            F_eval=F_new(I)
        return F_eval
    def square_JF_int(I): #Interval enclousre of the Jacobian of the polynomial system F.
        JF=sp.lambdify(x,sp.Matrix([F]).jacobian([x]),'numpy')
        if isinstance(I,np.ndarray):
            JF_eval=np.array(JF(*I))
        else:
            JF_eval=JF(I)[0]
        return JF_eval
    def JF_inverse(x_sol): #Take inverse of Jacobian and make compatible with inttervals.
        JF=sp.lambdify(x,sp.Matrix([F]).jacobian([x]),'numpy')
        JF_eval=JF(*x_sol[:,0])
        if len(JF_eval)>1:
            JF_inv=np.linalg.inv(JF_eval)
            JF_inv_int=np.zeros((len(JF_inv),len(JF_inv)),dtype=object)
            for i in range(len(JF_inv)):
                for j in range(len(JF_inv)):
                    re=JF_inv[i,j].real
                    im=JF_inv[i,j].imag
                    JF_inv_int[i,j]=interval([re,re],[im,im])
            JF_inv=JF_inv_int
        else:
            JF_inv=interval([1,1],[0,0])/interval(JF_eval[0][0][0].real,JF_eval[0][0][0].imag)
        return JF_inv
    def Interval_func(A_matrix,solution,solution_enclosure,unit_roundoff=u): #Function that finds an interval around the solutions, which we hope to be a strong interval approximate zero.
        AF=matrix_vector_prod(A_matrix,F_int(solution_enclosure))
        Int=np.zeros((len(AF)),dtype=object)
        if isinstance(AF,np.ndarray):
            for i in range(len(Int)): #Iterate over coordinates:
                absolute_AFi=(max(abs(float((AF[i].real).a)),abs(float((AF[i].real).b)))+max(abs(float((AF[i].imag).a)),abs(float((AF[i].imag).b))))**(1/2) #Find |A*\squareF(I)_j|
                re_low=solution[i].real[0]-absolute_AFi*unit_roundoff**(-1/4) #Add and subtract this estimate from the endpoints of real and imaginary parts of intevals multplied by u^(-1/4)
                re_high=solution[i].real[0]+absolute_AFi*unit_roundoff**(-1/4)
                im_low=solution[i].imag[0]-absolute_AFi*unit_roundoff**(-1/4)
                im_high=solution[i].imag[0]+absolute_AFi*unit_roundoff**(-1/4)
                Int[i]=interval([re_low,re_high],[im_low,im_high])
        else: #Same in one dimension.
            absolute_AFi=max(abs(float((AF.real).a)),abs(float((AF.real).b)))+max(abs(float((AF.imag).a)),abs(float((AF.imag).b)))**(1/2)        
            Int=np.array([iv.mpc([solution.real-absolute_AF*unit_roundoff**(-1/4),solution.real+absolute_AF*unit_roundoff**(-1/4)],[solution.imag-absolute_AF*unit_roundoff**(-1/4),solution.imag+absolute_AF*unit_roundoff**(-1/4)])])
        return Int
    intervallist=[]
    accuracylist=[]
    cert_list=[]
    distinct_solutions=0
    certified_solutions=0
    real_solutions_min=0
    real_solutions_max=len(sol)
    #Iterate over provided solutions:
    for solution in sol:
        v=1
        Sol_x=np.zeros(len(solution),dtype=object)
        for i in range(len(solution)):
            Sol_x[i]=iv.mpc([(solution[i].real)[0],(solution[i].real)[0]],[(solution[i]).imag[0],(solution[i].imag)[0]]) #Create a small interval enclosure around the solution.
        Sol_x_enclosure=Sol_x 
        A_matrix=JF_inverse(solution) #Set invertible matrix to be inverse of jacobian of F.
        Interval=Interval_func(A_matrix,solution,Sol_x_enclosure,u) #Define the interval using the previously stated function.
        #Krawczyk test:
        approx,unique,real,accuracy=Krawczyk(F_int,Interval,A_matrix,Sol_x,square_JF_int)
        #If Krawczyk test fails, we create a large interval by multiplying the unit roundoff by an increasingly larger number.
        while unique==False and v<10**10: #We try for a maximum of 10 different intervals where we multiply u by 10 each time.
            v=10*v
            Interval=Interval_func(A_matrix,solution,Sol_x_enclosure,v*u)
            approx,unique,real,accuracy=Krawczyk(F_int,Interval,A_matrix,Sol_x,square_JF_int)
        accuracylist.append(accuracy)
        #If Krawczyk test is sucessful, we have a uniqer solution.
        if unique is True:
            certified_solutions+=1 #Increase number of certified solutions by 1.
            if real is True: #Increase minimal number of real solutions by 1
                real_solutions_min+=1
            elif real is False: #Decrease maximal nuber of real solution by 1.
                real_solutions_max-=1
            intervallist.append(Interval) #Keep the intervals in a list
            cert_list.append([solution,f"Real: {real}"]) #Keep certified solutions and whether they are real.

    for i in range(len(intervallist)): #Checks all intervals
        I=intervallist[i]
        for j in range(len(intervallist)): #Compare with other intervals.
            J=intervallist[j]        
            if j!=i: #Ensures we do not compare the interval with itself.
                if any(I[k].real.b<J[k].real.a or I[k].real.a>J[k].real.b or I[k].imag.b<J[k].imag.a or I[k].imag.a>J[k].imag.b for k in range(len(I))): #Checks if there is any overlap with other intervals.
                    overlap=False
                else:
                    overlap=True
                    break #If interval overlap with any other interval, we break the loop, and claim the solution is not distinct.
        if overlap is False: #If strong interval approximate zero of the solution does not overlap, then it is distinct.
            distinct_solutions+=1
            cert_list[i]=np.append(cert_list[i],"Distinct")
            cert_list[i]=tuple(cert_list[i])
        if overlap is True: #If strong interval approximate zero of the solution does overlap with another interval, then it is not distinct.
            cert_list[i]=np.append(cert_list[i],"Not distinct")
            cert_list[i]=tuple(cert_list[i])
    if real_solutions_min==real_solutions_max:
        if distinct_solutions==certified_solutions:
            return f"Certified solutions: {certified_solutions} (total: {len(sol)}), Exactly {real_solutions_min} real solutions, Exactly {distinct_solutions} distinct solutions, Maximal real and imaginary inaccuracy: {max(accuracylist)}", cert_list
        else:
            return f"Certified solutions: {certified_solutions} (total: {len(sol)}), Exactly {real_solutions_min} real solutions, Minimum {distinct_solutions} distinct solutions, Maximal real and imaginary inaccuracy: {max(accuracylist)}", cert_list

    else:
        if distinct_solutions==certified_solutions:
            return f"Certified solutions: {certified_solutions} (total: {len(sol)}), Between {real_solutions_min} and {real_solutions_max} real solutions, Exactly {distinct_solutions} distinct solutions, Maximal real and imaginary inaccuracy: {max(accuracylist)}", cert_list        
        else:
            return f"Certified solutions: {certified_solutions} (total: {len(sol)}), Between {real_solutions_min} and {real_solutions_max} real solutions, Minimum {distinct_solutions} distinct solutions, Maximal real and imaginary inaccuracy: {max(accuracylist)}", cert_list